# Homework 2
### COSC 594 Bioinformatics
### Started 01/29/18, Due 02/06/18 
### Elliot Greenlee

1\. See included files, fruit_fly.fasta and human.fasta

In [1]:
# Input a genome from a file in fasta format
def input_genome(filename, chosen_gen):    
    with open(filename, 'r') as f:
        header = f.readline()
        current_gen = 1
        genome = ""
        for line in f:
            if line[0] is '>':
                if current_gen is chosen_gen:
                    genome = genome.replace('\n', '')
                    genome = genome.replace('N', 'A')
                    return header, genome
                else:
                    header = line
                    current_gen += 1
            else:
                genome = genome + line
          
        if current_gen is chosen_gen:
            genome = genome.replace('\n', '')
            genome = genome.replace('N', 'A')
            return header, genome
        else:
            print("This file only has {} genomes".format(current_gen))
            exit(1)

In [2]:
# 2. This program, globalign, computes the global alignment score of the human and fruit fly sequences. 
# Current parameters are match: +2, mismatch: -1, and gap: -2. 
# This program only calculates one of the possible best tracebacks, prioritizing left, diagonal, then up

import numpy as np

def globalign(sequence1, sequence2, match, mismatch, gap):
    # Initialize tables
    scores = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype=int)
    traceback = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype='i,i')
    
    # Add starting scores and traceback
    traceback[0][0] = (-1, -1)

    # First row
    for c in range(len(scores[0])):
        scores[0][c] = gap * c
        
        if c is not 0:
            traceback[0][c] = (0, c-1)
    
    # First column
    for r in range(len(scores)):
        scores[r][0] = gap * r
        
        if r is not 0:
            traceback[r][0] = (r-1, 0)
        
    # Calculate scores and traceback, row by row
    for r in range(1, len(scores)):
        for c in range(1, len(scores[0])):
            # Calculate the three possible scores
            gap2sequence1 = scores[r-1][c] + gap # Look up
            gap2sequence2 = scores[r][c-1] + gap # Look left
            
            if sequence1[r-1] is sequence2[c-1]:  # Look diagonal
                sequence2sequence = scores[r-1][c-1] + match
            else:
                sequence2sequence = scores[r-1][c-1] + mismatch
                
            # Compare scores to find the largest and the correct traceback(s)
            if gap2sequence2 >= gap2sequence1 and gap2sequence2 >= sequence2sequence:
                # left
                scores[r][c] = gap2sequence2
                traceback[r][c] = (r, c-1)
            elif gap2sequence1 >= gap2sequence2 and gap2sequence1 >= sequence2sequence:
                # up
                scores[r][c] = gap2sequence1
                traceback[r][c] = (r-1, c)
            elif sequence2sequence >= gap2sequence1 and sequence2sequence >= gap2sequence2:
                # diagonal
                scores[r][c] = sequence2sequence
                traceback[r][c] = (r-1, c-1)
    
    r = len(sequence1)
    c = len(sequence2)
    trace = traceback[r][c]
    rev_seq1 = ""
    rev_seq2 = ""
    
    while r != 0 or c != 0:
        r_diff = trace[0] - r
        c_diff = trace[1] - c
        
        if r_diff == -1 and c_diff == -1:
            # diagonal trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "{}".format(sequence2[c-1])
            r += -1
            c += -1
        elif r_diff == -1:
            # up trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "-"
            r += -1
        elif c_diff == -1:
            # left trace
            rev_seq1 += "-"
            rev_seq2 += "{}".format(sequence2[c-1])
            c += -1
            
        trace = traceback[r][c]
    
    seq1_align = rev_seq1[::-1]
    seq2_align = rev_seq2[::-1]
    align_score = scores[-1][-1]
        
    return align_score, seq1_align, seq2_align
    
    
# Constants
match = 2
mismatch = -1
gap = -2

# Read in the sequences
header, fruit_fly = input_genome("fruit_fly.fasta", 1)
header, human = input_genome("human.fasta", 1)

align_score, fruit_fly_align, human_align = globalign(fruit_fly, human, match, mismatch, gap)

print("Score: {}".format(align_score))
print()

for i in range(0, len(fruit_fly_align), 100):
    print("fruit fly " + fruit_fly_align[i:i+100])
    print("human     " + human_align[i:i+100])
    print()

Score: -1016

fruit fly TTCGCACGGCGTGCGTTTGGCTGAACACAGCAGTCTCTTGGCTAAAGCTTTCATGAGCAGTGCATGTAATAAAAACTGAGATCCAACTATGTTTACATTG
human     AT-GCA-G---------------AACA--G---TCAC------A--GC------G-G-AGTG-A---A-T-----C--AG--C----T-------C---G

fruit fly CAACCAACTCCAACTGCTATAGGCACCGTGGTTCCCCCATGGTCAGCGGGAACATTGATAGAGCGCCTGCCGTCTTTAGAAGACATGGCTCACAAGGGTC
human     ---------------G-T---GGT---GTC-TT------TG-TCAACGGG--C---G---GC-CAC-TGCCG------GA---C-T--C-CAC------C

fruit fly ACAGTGGAGTAAATCAGCTGGGTGGCGTTTTTGTTGGAGGAAGGCCTTTGCCAGATTCAACACGGCAAAAAATTGTCGAACTGGCACATTCTGGAGCTCG
human     -C-G-GCAG-AAG--A--T---TG---T---------AG-A-G-C-TA-GC----T-CA-CA-G-C--------G--G----GGC-C---C-GG--C-CG

fruit fly GCCATGTGATATTTCTCGAATTCTGCAAGTATCAAATGGATGTGTGAGCAAAATTCTCGGG-AGGTATTATGAAACAGGAAGCATACGACCACGTGCTAT
human     ----TGCGACATTTCCCGAATTCTGCAGGTGTCCAACGGATGTGTGAGTAAAATTCT-GGGCAGGTATTACGAGACTGG---C-T-C--C-A--T-C-A-

fruit fly CGGAGGATCCAAGCCACGTGTGGCCACAGCCGAAGTCGTTAGCAAAATTTCGCAGTACAAACGCGAGTGTCCTAGCATATTTGC

In [3]:
# 3. This program, localign, computes the local alignment score of the human and fruit fly sequences. 
# Current parameters are match: +2, mismatch: -1, and gap: -2. 
# This program only calculates one of the possible best tracebacks, prioritizing left, diagonal, then up

import numpy as np

def localign(sequence1, sequence2, match, mismatch, gap):
    # Initialize tables
    scores = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype=int)
    traceback = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype='i,i')
    
    # Add starting scores and traceback
    # First row
    for c in range(len(scores[0])):
        scores[0][c] = 0
        traceback[0][c] = (-1, -1)
    
    # First column
    for r in range(len(scores)):
        scores[r][0] = 0
        traceback[r][0] = (-1, -1)
            
    best_r = 0
    best_c = 0
    best_score = 0
        
    # Calculate scores and traceback, row by row
    for r in range(1, len(scores)):
        for c in range(1, len(scores[0])):
            # Calculate the three possible scores
            gap2sequence1 = scores[r-1][c] + gap # Look up
            gap2sequence2 = scores[r][c-1] + gap # Look left
            
            if sequence1[r-1] is sequence2[c-1]:  # Look diagonal
                sequence2sequence = scores[r-1][c-1] + match
            else:
                sequence2sequence = scores[r-1][c-1] + mismatch
                
            # Compare scores to find the largest and the correct traceback(s)
            if gap2sequence1 >= gap2sequence2 and gap2sequence1 >= sequence2sequence:
                # up
                scores[r][c] = gap2sequence1
                traceback[r][c] = (r-1, c)
            elif sequence2sequence >= gap2sequence1 and sequence2sequence >= gap2sequence2:
                # diagonal
                scores[r][c] = sequence2sequence
                traceback[r][c] = (r-1, c-1)
            elif gap2sequence2 >= gap2sequence1 and gap2sequence2 >= sequence2sequence:
                # left
                scores[r][c] = gap2sequence2
                traceback[r][c] = (r, c-1)
                
            if scores[r][c] < 0:
                scores[r][c] = 0
                traceback[r][c] = (-1, -1)
                
            if scores[r][c] >= best_score:
                best_score = scores[r][c]
                best_r = r
                best_c = c
    
    r = best_r
    c = best_c
    trace = traceback[r][c]
    rev_seq1 = ""
    rev_seq2 = ""
    
    while trace[0] != -1 or trace[1] != -1:
        r_diff = trace[0] - r
        c_diff = trace[1] - c
        
        if r_diff == -1 and c_diff == -1:
            # diagonal trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "{}".format(sequence2[c-1])
            r += -1
            c += -1
        elif r_diff == -1:
            # up trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "-"
            r += -1
        elif c_diff == -1:
            # left trace
            rev_seq1 += "-"
            rev_seq2 += "{}".format(sequence2[c-1])
            c += -1
            
        trace = traceback[r][c]
    
    seq1_align = rev_seq1[::-1]
    seq2_align = rev_seq2[::-1]
    align_score = best_score
        
    return align_score, seq1_align, seq2_align
    
    
# Constants
match = 2
mismatch = -1
gap = -2

# Read in the sequences
header, fruit_fly = input_genome("fruit_fly.fasta", 1)
header, human = input_genome("human.fasta", 1)

align_score, fruit_fly_align, human_align = localign(fruit_fly, human, match, mismatch, gap)

print("Score: {}".format(align_score))
print()

for i in range(0, len(fruit_fly_align), 100):
    print("fruit fly " + fruit_fly_align[i:i+100])
    print("human     " + human_align[i:i+100])
    print()

Score: 820

fruit fly ATGGCTCACAAGGGTCACAGTGGAGTAAATCAGCTGGGTGGCGTTTTTGTTGGAGGAAGGCCTTTGCCAGATTCAACACGGCAAAAAATTGTCGAACT-G
human     ATG-C--AGAACAGTCACAGCGGAGTGAATCAGCTCGGTGGTGTCTTTGTCAACGGGCGGCCACTGCCGGACTCCACCCGGCAGAAGATTGTAGAGCTAG

fruit fly -GCACATTCTGGAGCTCGGCCATGTGATATTTCTCGAATTCTGCAAGTATCAAATGGATGTGTGAGCAAAATTCTCGGGAGGTATTATGAAACAGGAAGC
human     CTCACAG-CGGG-GCCCGGCCGTGCGACATTTCCCGAATTCTGCAGGTGTCCAACGGATGTGTGAGTAAAATTCTGGGCAGGTATTACGAGACTGGCTCC

fruit fly AT-ACGA-CCACGTGCTATCGGAGG-A-TCCAAGCCACGTGTGGCCACAGCCGAAGTCGTTAGCAAAATTTCGCAGTACAAACGCGAGTGTCC-TAGCAT
human     ATCA-GACCCAGG-GCAATCGGTGGTAGTA-AACCGA-GAGTAGCGACTCCAGAAGTTGTAAGCAAAATAGCCCAGTATAAGCGGGAGTGCCCGTC-CAT

fruit fly ATTTGCTTGGGAAATTCGGGATAGATTACT-TCAGGAGAACGTTTGTACTAACGATAATATACCAAGTGTGTCCTCAATAAACCGTG-TATTGAGAAACT
human     CTTTGCTTGGGAAATCCGAGACAGATTACTGTCCG-AGGGGGTCTGTACCAACGATAACATACCAAGCGTGTCATCAATAAACAGAGTTCTTC-GCAACC

fruit fly TGGCT-GCGCAAAAGGAGCAGCAAAGCACGGGATCCGGGA-GCTCCAGCACATCCGCCGGCAACTCAATCAGCGCAAAAGTGTCTG

3\. Discussion. After viewing the global alignment, it did not seem these two sequences were related. However, the local alignment seems to make it clear that a large portion of the two sequences are very similar. The matching portion for local alignment runs from 184 to 1489 in the fruit fly sequence, and from 0 to 1267 in the human sequence.

In [4]:
# 4. See included files, human_mito.fasta and neander_sample.fasta
# This program, egfalign, computes the end-gap free alignment score of the human mitochondrial and neanderthal sequences. 
# Current parameters are match: +2, mismatch: -1, and gap: -2. 
# This program only calculates one of the possible best tracebacks, prioritizing left, diagonal, then up

import numpy as np

def egfalign(sequence1, sequence2, match, mismatch, gap):
    # Initialize tables
    scores = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype=int)
    traceback = np.zeros((len(sequence1) + 1, len(sequence2) + 1), dtype='i,i')
    
    # Add starting scores and traceback
    # First row
    for c in range(len(scores[0])):
        scores[0][c] = 0 
        traceback[0][c] = (-1, -1)
    
    # First column
    for r in range(len(scores)):
        scores[r][0] = 0
        traceback[r][0] = (-1, -1)
            
    best_r = 0
    best_c = 0
    best_score = 0
        
    # Calculate scores and traceback, row by row
    for r in range(1, len(scores)):
        for c in range(1, len(scores[0])):
            # Calculate the three possible scores
            gap2sequence1 = scores[r-1][c] + gap # Look up
            gap2sequence2 = scores[r][c-1] + gap # Look left
            
            if sequence1[r-1] is sequence2[c-1]:  # Look diagonal
                sequence2sequence = scores[r-1][c-1] + match
            else:
                sequence2sequence = scores[r-1][c-1] + mismatch
                
            # Compare scores to find the largest and the correct traceback(s)
            if gap2sequence1 >= gap2sequence2 and gap2sequence1 >= sequence2sequence:
                # up
                scores[r][c] = gap2sequence1
                traceback[r][c] = (r-1, c)
            elif sequence2sequence >= gap2sequence1 and sequence2sequence >= gap2sequence2:
                # diagonal
                scores[r][c] = sequence2sequence
                traceback[r][c] = (r-1, c-1)
            elif gap2sequence2 >= gap2sequence1 and gap2sequence2 >= sequence2sequence:
                # left
                scores[r][c] = gap2sequence2
                traceback[r][c] = (r, c-1)
                
            # keep track of the best score in the last row and column
            if ((r == len(scores)-1 ) or (c == len(scores[0])-1)) and scores[r][c] >= best_score:
                best_score = scores[r][c]
                best_r = r
                best_c = c
    
    r = best_r
    c = best_c
    trace = traceback[r][c]
    rev_seq1 = ""
    rev_seq2 = ""
    
    while trace[0] != -1 or trace[1] != -1:
        r_diff = trace[0] - r
        c_diff = trace[1] - c
        
        if r_diff == -1 and c_diff == -1:
            # diagonal trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "{}".format(sequence2[c-1])
            r += -1
            c += -1
        elif r_diff == -1:
            # up trace
            rev_seq1 += "{}".format(sequence1[r-1])
            rev_seq2 += "-"
            r += -1
        elif c_diff == -1:
            # left trace
            rev_seq1 += "-"
            rev_seq2 += "{}".format(sequence2[c-1])
            c += -1
            
        trace = traceback[r][c]
    
    seq1_align = rev_seq1[::-1]
    seq2_align = rev_seq2[::-1]
    align_score = best_score
        
    return align_score, seq1_align, seq2_align
    
    
# Constants
match = 2
mismatch = -1
gap = -2

# Read in the sequences
header, human = input_genome("human_mito.fasta", 1)
header, neanderthal = input_genome("neander_sample.fasta", 1)

align_score, human_align, neanderthal_align = egfalign(human, neanderthal, match, mismatch, gap)

print("Score: {}".format(align_score))
print()

for i in range(0, len(human_align), 100):
    print("human       " + human_align[i:i+100])
    print("neanderthal " + neanderthal_align[i:i+100])
    print()

Score: 620

human       CCAAGTATTGACTCACCCATCAACAACCGCTATGTATTTCGTACATTACTGCCAGCCACCATGAATATTGTACGGTACCATAAATACTTGACCACCTGTA
neanderthal CCAAGTATTGACTCACCCATCAACAACCGCCATGTATTTCGTACATTACTGCCAGCCACCATGAATATTGTACAGTACCATAATTACTTGACTACCTGTA

human       GTACATAAAAACCCAATCCACATCAAAACCCCCTCCCCATGCTTACAAGCAAGTACAGCAATCAACCCTCAACTATCACACATCAACTGCAACTCCAAAG
neanderthal ATACATAAAAACCTAATCCACATCAACCCCCCCCCCCCATGCTTACAAGCAAGCACAGCAATCAACCTTCAACTGTCATACATCAACTACAACTCCAAAG

human       CCACCCCT-CACCCACTAGGATACCAACAAACCTACCCACCCTTAACAGTACATAGTACATAAAGCCATTTACCGTACATAGCACATTACAGTCAAATCC
neanderthal ACACCCTTACACCCACTAGGATATCAACAAACCTACCCACCCTTGACAGTACATAGCACATAAAGTCATTTACCGTACATAGCACATTATAGTCAAATCC

human       CTTCTCGTCCCCATGGATGACCCCCCTCAGATAGGGGTCCCTTGA
neanderthal CTTCTCGCCCCCATGGATGACCCCCCTCAGATAGGGGTCCCTTGA



5\. Discussion. Similarly to the local alignment calculated, the fruit fly alignment starts around 200 and ends around 1500, and the human alignment starts around 0, but ends much sooner around 800. This seems to be related to the blast algorithm choosing two separate local alignments rather than one single local alignment, and using a different scoring system. Despite the differences, it seems to confirm the local alignment results. The first of the two alignments is of length 397, identity 73%, and e-value 4e-65. The second of the two alignments is of length 203, identity 71%, and e-value 6e-25.

6\. The "mystery" sequence has 9410 nucleotides, and a label of gi|224589819:c155604967-155595558 Homo sapiens chromosome 7, GRCh37.p9 Primary Assembly. 

a. blastn: The mystery sequence has 244 blast hits on 100 sequences. Three of these are an exact match to the query, with 100% cover and 100% identity. Homo sapiens sonic hedgehog (SHH), RefSeqGene on chromosome 7; Homo sapiens BAC clone RP11-69O3 from 7, complete sequence; and Human cosmid clone LL157A05 from 7q36, complete sequence. 

b. blastx: The mystery sequence has 252 blast hits on 100 sequences. The highest coverage is 15% by 8 sequences, and three of these have the highest identify of 99%. sonic hedgehog homolog (Drosophila) [Homo sapiens]; PREDICTED: sonic hedgehog protein [Pan troglodytes]; and PREDICTED: sonic hedgehog protein isoform X1 [Gorilla gorilla gorilla]

In [5]:
# 7. Linear space # 4

import numpy as np

def egfalign_linear(sequence1, sequence2, match, mismatch, gap):
    # Set sequence2 to be the shorter of the two sequences
    if len(sequence2) > len(sequence1):
        tmp = sequence2
        sequence2 = sequence1
        sequence1 = tmp
    
    # Initialize score rows
    scores = np.zeros((2, len(sequence2) + 1), dtype=int)
    
    # Add starting scores
    # First row
    for c in range(len(scores[0])):
        scores[0][c] = 0 
            
    best_score = 0
        
    # Calculate scores and traceback, row by row
    for r in range(0, len(sequence1)):
        # Figure out first and second by modding r
        first = r % 2
        second = (r + 1) % 2
        
        # set first column in the second row to 0
        scores[second][0] = 0
        
        # change all rs to first or second
        for c in range(1, len(sequence2) + 1):
            # Calculate the three possible scores
            gap2sequence1 = scores[first][c] + gap # Look up
            gap2sequence2 = scores[second][c-1] + gap # Look left
            
            if sequence1[r-1] is sequence2[c-1]:  # Look diagonal
                sequence2sequence = scores[first][c-1] + match
            else:
                sequence2sequence = scores[first][c-1] + mismatch
                
            # Compare scores to find the largest and the correct traceback(s)
            if gap2sequence1 >= gap2sequence2 and gap2sequence1 >= sequence2sequence:
                # up
                scores[second][c] = gap2sequence1
            elif sequence2sequence >= gap2sequence1 and sequence2sequence >= gap2sequence2:
                # diagonal
                scores[second][c] = sequence2sequence
            elif gap2sequence2 >= gap2sequence1 and gap2sequence2 >= sequence2sequence:
                # left
                scores[second][c] = gap2sequence2
                
            # keep track of the best score in the last row and column
            if ((r == len(sequence1)-1) or (c == len(sequence2))) and scores[second][c] >= best_score:
                best_score = scores[second][c]
                
        # set first row = second
        scores[first] = scores[second]
    
    align_score = best_score
        
    return align_score
    
    
# Constants
match = 2
mismatch = -1
gap = -2

# Read in the sequences
header, human = input_genome("human_mito.fasta", 1)
header, neanderthal = input_genome("neander_sample.fasta", 1)

align_score = egfalign_linear(human, neanderthal, match, mismatch, gap)

print("Score: {}".format(align_score))

Score: 620
